# Graph Classification

## Graph Classes

we use the following graph classes:
- Chemical Graphs (Molecules)
- Random
- Small World
- Scale Free

## Setup

We use pyg (pytorch-geometric) to generate the model to train.
The model is a GCN with 2 layers and 32 hidden units.

In [ ]:
import torch_geometric
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch_geometric.__version__)

In [24]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx

# we need to read the dataset from the pickle file
import pickle

with open('dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

for key in dataset.keys():
    print(f"datasetkey: {key}, shape: {len(dataset[key])}")

# we need to convert the networkx graphs to pytorch geometric graphs

class_key = {
    'random': 0,
    'smallworld': 1,
    'scalefree': 2,
    'chemical': 3
}
pyg_dataset = []
for key in dataset.keys():
    for i in range(len(dataset[key])):
        graph = dataset[key][i]
        graph_tensor = from_networkx(graph)
        graph_tensor.y = torch.tensor([class_key[key]])
        graph_tensor.x = graph_tensor.x.view(-1, 1)
        graph_tensor.x = graph_tensor.x.to(torch.float)
        pyg_dataset.append(graph_tensor)

datasetkey: random, shape: 40
datasetkey: smallworld, shape: 40
datasetkey: scalefree, shape: 40
datasetkey: chemical, shape: 0


In [39]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx
import pickle


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']


    def process(self):
        # Read data into huge `Data` list.
        with open('dataset.pickle', 'rb') as f:
            dataset = pickle.load(f)

        class_key = {
            'random': 0,
            'smallworld': 1,
            'scalefree': 2,
            'chemical': 3
        }
        data_list = []
        for key in dataset.keys():
            for i in range(len(dataset[key])):
                graph = dataset[key][i]
                graph_tensor = from_networkx(graph)
                graph_tensor.y = torch.tensor([class_key[key]])
                graph_tensor.x = graph_tensor.x.view(-1, 1)
                graph_tensor.x = graph_tensor.x.to(torch.float)
                data_list.append(graph_tensor)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [40]:
import torch
from torch_geometric.datasets import TUDataset

dataset = MyOwnDataset(root='data/CustomData')

# dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...



Dataset: MyOwnDataset(3375):
Number of graphs: 3375
Number of features: 1
Number of classes: 3

Data(x=[5, 1], edge_index=[2, 16], y=[1])
tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.]])
Number of nodes: 5
Number of edges: 16
Average node degree: 3.20
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


In [43]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:2700]
test_dataset = dataset[2700:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 2700
Number of test graphs: 675


In [44]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[1675, 1], edge_index=[2, 22064], y=[64], batch=[1675], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[1814, 1], edge_index=[2, 29478], y=[64], batch=[1814], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[1745, 1], edge_index=[2, 22796], y=[64], batch=[1745], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[1948, 1], edge_index=[2, 24864], y=[64], batch=[1948], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[1649, 1], edge_index=[2, 23220], y=[64], batch=[1649], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[1788, 1], edge_index=[2, 24906], y=[64], batch=[1788], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[1707, 1], edge_index=[2, 26424], y=[64], batch=[1707], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[1805, 1], edge_index=[2, 25258], y

In [45]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=3, bias=True)
)


In [46]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.3674, Test Acc: 0.3496
Epoch: 002, Train Acc: 0.3748, Test Acc: 0.3378
Epoch: 003, Train Acc: 0.3519, Test Acc: 0.3644
Epoch: 004, Train Acc: 0.3415, Test Acc: 0.3570
Epoch: 005, Train Acc: 0.3452, Test Acc: 0.3481
Epoch: 006, Train Acc: 0.3504, Test Acc: 0.3541
Epoch: 007, Train Acc: 0.3496, Test Acc: 0.3422
Epoch: 008, Train Acc: 0.3607, Test Acc: 0.3467
Epoch: 009, Train Acc: 0.3570, Test Acc: 0.3600
Epoch: 010, Train Acc: 0.3670, Test Acc: 0.3556
Epoch: 011, Train Acc: 0.3670, Test Acc: 0.3541
Epoch: 012, Train Acc: 0.3367, Test Acc: 0.3200
Epoch: 013, Train Acc: 0.3567, Test Acc: 0.3719
Epoch: 014, Train Acc: 0.3578, Test Acc: 0.3674
Epoch: 015, Train Acc: 0.3796, Test Acc: 0.3719
Epoch: 016, Train Acc: 0.3411, Test Acc: 0.3689
Epoch: 017, Train Acc: 0.3748, Test Acc: 0.3659
Epoch: 018, Train Acc: 0.4067, Test Acc: 0.3778
Epoch: 019, Train Acc: 0.4181, Test Acc: 0.3896
Epoch: 020, Train Acc: 0.3815, Test Acc: 0.3615
Epoch: 021, Train Acc: 0.4059, Test Acc: